<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/01_bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストのBoW (bag-of-words)モデル
* テキストを単語のmultisetとして表現したもの
  * もはや単語の出現回数でしかテキストを区別できない。
  * つまり、語順は無視される。
  * それでも、それなりに興味深い分析が実現できる。


## 前置き

### 参考書

* 授業全体の参考書: 岡﨑、荒瀬、鈴木、鶴岡、宮尾著 『IT Text 自然言語処理の基礎』（オーム社）
  * https://www.ohmsha.co.jp/book/9784274229008/
* 本日の参考書: C. D. Manning, P. Raghavan & H. Schütze. Introduction to Information Retrieval.
  * https://nlp.stanford.edu/IR-book/html/htmledition/irbook.html

### prerequisites

* この授業では、Pythonのコーディングの基礎は習得済みであることを前提します。
* また、NumPyやscikit-learnの基本的な使い方は習得済みであることを前提します。
* PyTorch（Hugging Faceのライブラリを使う時に必要）は、授業で説明します。

### NLPの歴史
* bag-of-words(BoW)は、テキストをモデル化する方法の、一つ。
* BoWから文脈化埋め込み(contextualized word embeddings)に至る流れを把握することが、この授業での学習の目標のひとつ。
  * LLMs (large language models) は、文脈化埋め込みを実現する手法の一つ。
* NLPの歴史の参考資料
  * スタンフォード大のNLPの授業がいかに大きく内容を変えているか、調べてみよう。
  * http://web.stanford.edu/class/cs224n/index.html の"Previous offerings"

### 用語集

* **単語(word)**
  * テキストを構成する最小の単位。
  * LLMの世界では、単語をさらに分割したサブワード(subword)が最小の単位になる。

* **語彙(vocabulary)**
  * あるコーパスに出現する単語またはサブワードの集合のこと。


* **トークン(token)**
  * 単語(word)、またはサブワード(subword)の、一回一回の出現のこと。
  * 例えば、このセルで「この」という単語は5回現れている。
  * このことを、このセルでは「この」という単語のトークンが5個ある、と言い表す。


* **テキスト(text)**
  * トークン列のこと。
  * 当然、同じ単語のトークンが複数回現れることもある。
  * テキストのことを、文書(document)と呼ぶこともある。
* **コーパス(corpus)**
  * テキストの集合のこと。

## テキストのBoW(bag-of-words)モデル
* **bag-of-wordsモデル**とは、テキストを定量的に表現する手法のひとつ。
  * 他にもテキストを定量的に表現する手法はある。
* bag-of-wordsモデルにおいては、トークンの**出現順序が無視される**。
* つまり、テキストを、バッグに入ったトークンの集まりのようにモデリングする（下図参照）。
  * 言い換えれば、テキストを単語の**multiset**として扱うのがbag-of-wordsモデルである。


![bag-of-words.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/bag-of-words.png)

* 上の図は下記のWebページより。
 * https://dudeperf3ct.github.io/lstm/gru/nlp/2019/01/28/Force-of-LSTM-and-GRU/

### テキストのベクトル表現の歴史
* 最近では、単語またはsubwordのベクトル表現を利用して、テキストのベクトル表現を作る。
  * word2vec以降の流れ。
* 単語またはsubwordのベクトル表現は、埋め込み(embedding)と呼ばれる。
  * 分散表現(distributive representation)とも呼ばれるが、最近あまり使わない呼び方。
* word embeddingやsubword embeddingを元にして、テキストをembedするのが、今は主流。

### BoWはまだ現役か？
* 論文では今でも、baselineとして、TF-IDFやBM25など、BoWが引き合いに出されることがある。
  * 新しい手法を考え出してもBoWに勝てなければ意味がない。
* そのため、最初にBoWについて簡単に説明しておく。
  * BM25 https://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html
* ただし、最近でもRAGではBM25をdense retrievalと組み合わせることもあるので、まだ現役と言えるかも。
  * RAG = retrieval augmented generation

## BoWモデル1: Word count
* テキストは、各単語の出現回数を要素とするベクトルとして表現できる。

### scikit-learnのCountVectorizer
* 各テキストは、半角スペースでつながれた単語の列として準備しておく。
* CountVectorizerのインスタンスを作り、テキスト集合にfitさせる。
  * 語彙の抽出と、出現回数の集計が実行される。
* そして他の任意のテキスト集合をtransformする。
  * この使い方は、scikit-learnにおける他の前処理のときと同様。
  * fitメソッドに与えたテキスト集合の語彙が使われる。


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

* コーパス（＝テキストの集合）を用意する。

In [ ]:
corpus = [
    "This document is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Where is the fourth one?"
    ]

* CountVectorizerをデフォルトの設定で使う。

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

* テキストのBoW表現の確認
 * 疎なベクトルとして得られることに注意。

In [ ]:
print(X)

  (0, 9)	1
  (0, 1)	2
  (0, 4)	1
  (0, 7)	1
  (0, 2)	1
  (1, 9)	1
  (1, 1)	2
  (1, 4)	1
  (1, 7)	1
  (1, 6)	1
  (2, 9)	1
  (2, 4)	1
  (2, 7)	1
  (2, 0)	1
  (2, 8)	1
  (2, 5)	1
  (3, 4)	1
  (3, 7)	1
  (3, 5)	1
  (3, 10)	1
  (3, 3)	1


In [ ]:
type(X)

scipy.sparse._csr.csr_matrix

* 疎な表現を通常の密な表現（NumPyのndarray）にする。

In [ ]:
X_dense = X.toarray()
X_dense

array([[0, 2, 1, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 2, 0, 0, 1, 0, 1, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1]])

### 語彙を確認
* 先頭の大文字は自動的に小文字に変換されていることが分かる。
* ピリオドや疑問符は削除されている。

In [ ]:
vectorizer.vocabulary_

{'this': 9,
 'document': 1,
 'is': 4,
 'the': 7,
 'first': 2,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'where': 10,
 'fourth': 3}

In [ ]:
type(vectorizer.vocabulary_)

dict

* 語彙を取得

In [ ]:
vocab = vectorizer.get_feature_names_out()
vocab

array(['and', 'document', 'first', 'fourth', 'is', 'one', 'second', 'the',
       'third', 'this', 'where'], dtype=object)

### 新しいテキストをベクトルに変換
* sklearnでよくやるように、transformメソッドを使う。

In [ ]:
new_doc = ["This is the new document."]
new_vectors = vectorizer.transform(new_doc)
new_vectors.toarray()

array([[0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]])

* 新出の単語は無視される点に注意
  * 問： 上の例題で、どれが新出単語か？
* OoV (out-of-vocabulary) の問題
  * この問題は、NLPの世界では、超重要な問題。
  * 今は、サブワード(subword)の利用により、OoV問題を回避する。

## BoWモデル2: TF-IDF
* テキストをベクトル化する古典的な手法。
* TF-IDFは、TFとIDFの積である。

### TF (term frequency)
* TFとは、各々の単語がテキストのなかで出現する回数。word countと同義。
  * 出現回数を、そのテキストの長さで割ったものをTFと呼ぶこともある。
  * テキストの長さとは、テキストに含まれるトークンの総数。
* テキストのなかで頻出する単語ほどTFは大きくなる。

### IDF (inverse document frequency)
* IDFとは、DFの逆数。
* DFとは、ある単語が含まれるテキストの数。
  * ある単語が含まれるテキスト数を全テキスト数で割ったものをDFと呼ぶこともある。
* テキスト集合のなかで稀少な単語ほどIDFは大きくなる。

### TF-IDF (term frequency–inverse document frequency)
* TF-IDFは、TFとIDFの積。
* 積を求める前に、TFのルートもしくは対数をとったり、IDFのルートもしくは対数をとったりする。
  * 大きめの値が、効きすぎないようにする。
  * 対数をとるときは、ゼロの対数をとることにならないように、1を足したりする。

### TF-IDFの式の例

\begin{align}
x_{d,w} = \frac{n_{d,w}}{n_d} \cdot ( 1 + \log\frac{m}{m_w}) \tag{1}
\end{align}

where

 * $n_{d,w}$ is the frequency of the word $w$ in the document $d$,
 * $n_d$ is defined as $n_d \equiv \sum_w n_{d,w}$,
 * $m_w$ is the number of documents containing the word $w$, and
 * $m$ is the total number of documents.

### TF-IDFの式のバリエーション

* https://nlp.stanford.edu/IR-book/html/htmledition/document-and-query-weighting-schemes-1.html

![img462.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/img462.png)



### 式の選び方
* どの式の形がいいかは、downstream taskの性能で選ぶ。
* どんな場合でもこれが一番良い、という式は、ない。

### scikit-learnのTfidfVectorizer
* scikit-learnでTF-IDFの計算式がどうなっているかは下記を参照。
  * https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting

* デフォルトの設定を確認
  * https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

* 次のパラメータは、ちゃんと考えて設定した方が良い。
  * `max_df`, `min_df`, `stop_words`

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
X = tfidf_vectorizer.fit_transform(corpus).toarray()
X

array([[0.        , 0.74846041, 0.47466356, 0.        , 0.24769914,
        0.        , 0.        , 0.24769914, 0.        , 0.3029716 ,
        0.        ],
       [0.        , 0.74846041, 0.        , 0.        , 0.24769914,
        0.        , 0.47466356, 0.24769914, 0.        , 0.3029716 ,
        0.        ],
       [0.52898651, 0.        , 0.        , 0.        , 0.2760471 ,
        0.41705904, 0.        , 0.2760471 , 0.52898651, 0.33764523,
        0.        ],
       [0.        , 0.        , 0.        , 0.56199026, 0.29326983,
        0.44307958, 0.        , 0.29326983, 0.        , 0.        ,
        0.56199026]])

* テキストベクトルはL2ノルムが1となるように長さを変更されている。
  * TfidfVectorizer()のnormパラメータで変更可能。


In [ ]:
import numpy as np
np.linalg.norm(X, axis=1)

array([1., 1., 1., 1.])

* 語彙の取得

In [ ]:
vocab = tfidf_vectorizer.get_feature_names_out()
vocab

array(['and', 'document', 'first', 'fourth', 'is', 'one', 'second', 'the',
       'third', 'this', 'where'], dtype=object)

* 新しいテキストをベクトル化
  * 新出単語は無視される（OoV問題）。

In [ ]:
new_vectors = tfidf_vectorizer.transform(new_doc).toarray()
new_vectors

array([[0.        , 0.6284927 , 0.        , 0.        , 0.41599288,
        0.        , 0.        , 0.41599288, 0.        , 0.50881901,
        0.        ]])

* 各単語のIDF
  * IDFはそれぞれの単語について一意に決まる値。
  * テキストごとに求まる値ではない。
  * コーパスが変わると、IDFも変わる。

In [ ]:
tfidf_vectorizer.idf_

array([1.91629073, 1.51082562, 1.91629073, 1.91629073, 1.        ,
       1.51082562, 1.91629073, 1.        , 1.91629073, 1.22314355,
       1.91629073])

## BoWの応用

### テキスト間の類似度計算

* 内積による類似度計算


In [ ]:
np.dot(X, new_vectors[0])

array([0.83064177, 0.83064177, 0.40146757, 0.24399632])

* TfidfVectorizorのデフォルトの設定では、TF-IDFベクトルが長さ1にnormalizeされている。
* そのため、内積がコサイン類似度に一致する。

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(X, new_vectors)

array([[0.83064177],
       [0.83064177],
       [0.40146757],
       [0.24399632]])

* 問: テキストをベクトルとして表現する方法が分かった。これを使うと何ができるか？

## 演習: 20 newsgroups データセット
* テキスト分類手法の評価に使う、古典的なデータセット。

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups()
y_true = newsgroups.target

* 下記コードを参考にして、数値を全て「#NUMBER」という特殊な単語へ変換する。
 * https://scikit-learn.org/stable/auto_examples/bicluster/plot_bicluster_newsgroups.html#sphx-glr-auto-examples-bicluster-plot-bicluster-newsgroups-py

In [ ]:
def number_normalizer(tokens):
  return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

class NumberNormalizingVectorizer(TfidfVectorizer):
  def build_tokenizer(self):
    tokenizer = super().build_tokenizer()
    return lambda doc: list(number_normalizer(tokenizer(doc)))

In [ ]:
vectorizer = NumberNormalizingVectorizer(stop_words='english', min_df=5)

In [ ]:
X = vectorizer.fit_transform(newsgroups.data)

In [ ]:
print(vectorizer.get_feature_names_out()[:20])

['#NUMBER' '_0' '_4' '_5' '_6' '_7u' '_8' '__' '___' '____' '_____'
 '______' '_______' '________' '_________' '__________' '___________'
 '____________' '_____________' '______________']


In [ ]:
vocab = vectorizer.get_feature_names_out()
len(vocab)

23427

* 密(dense)な配列に変換する。

In [ ]:
X = X.toarray()

* 適当な二つのテキストの内積を求めてみる。

In [ ]:
np.dot(X[0], X[1])

0.021480506411432166

* 対応するクラスラベルを調べる。

In [ ]:
y_true[0], y_true[1]

(7, 4)

In [ ]:
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

# 本日の課題
* TF-IDFベクトルを使って、20 newsgroupsの任意のテキストについて、それと最も似ているテキストを10個返す関数を書こう。
* 10個のうち、元のテキストと同じクラスに属するテキストがいくつあるかを調べよう。
* 全てのテキストについて同じことを行ない、最も似ている上位10個のうち同じクラスのテキスト数の平均値を求めよう。